## Cowell Power Plant Project

Cowell is a coastal town on Franklin Harbor on the eastern side of the Eyre Peninsula, in South Australia 493 km by road from Adelaide. Franklin Harbor is a natural harbour with a channel to the sea just 100 metres wide.

This town is the major population centre of the District Council of Franklin Harbour, and the centre of an agricultural district, farming wheat and sheep. Fishing, and more recently, oyster farming has also been an important industry.

<img src="data/Cowell_Satellite.png" alt="drawing" width="400"/>

Cowell has recently attracted the attention of the South Australian government as a potential place to test a promising new technology that allows getting energy from the tides. Franklin Harbour has a narrow channel to the sea through which, every day, millions of liters of water flow in and out as the tide changes. The people in the town are quite excited with the idea of becoming a major producer of energy in SA and hosting the first tidal clean energy plant in Australia.

### Your task:

You are an engineer working at Tidesla, a new company that has developed a promising technology for building and installing underwater turbines. You have been tasked with finding the best location in Australia to test the technology and you are starting by Cowell. The most urgent matter is getting an estimate about how much energy this plant could generate. You have no previous experience working in tidal power plants but you have heard about Digital Earth Australia and the InterTidal Extents Model (ITEM) product, which presents information of tidal extents in Australia observed from satellites.

Using this dataset you are going to calculate how many liters of water go through the channel every time there is a change of tide. These numbers can then be transformed into KWatts using the efficiency values provided by the turbine development team. 

One of your colleagues has given you the polygon delimiting Franklin Harbour. This polygon cuts the channels at the place where the underwater turbines would be placed. Go to [geojson.io](http://geojson.io) to visualise the polygon. 

> Tip: Paste the contents delimited by the outer curly braces, in the next cell, into the right hand side panel of geojson.io webpage.

In [ ]:
poly = """{
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "LineString",
        "coordinates": [[136.91986083984375,-33.66549665763364],
          [136.86492919921872,-33.71577279930021],
          [136.84432983398438,-33.75403149821638],
          [136.84398651123047,-33.798835278253975],
          [136.86458587646484,-33.79312909288506],
          [136.96208953857422,-33.745467600106174],
          [136.96964263916016,-33.738615865848736],
          [136.9552230834961,-33.73490428127589],
          [136.98612213134766,-33.693780606387506],
          [136.97650909423828,-33.667211101197545],
          [136.91986083984375,-33.66549665763364]]}}"""

### Getting ready by loading some packages 

Once you have verified the polygon data you decide to load your favorite Python libraries to start working on this project.

In [ ]:
%matplotlib inline

import datacube
import matplotlib.pyplot as plt
import numpy as np
import json
import sys
import xarray as xr

sys.path.append("./Scripts")
from dea_masking import mask

In [ ]:
dc = datacube.Datacube(app="tidal_powerplant")

### Selecting and loading the data

Getting Cowell's coordinates from Wikipedia you define an extension that covers the entire area around Franklin Harbour.

In [ ]:
y = -33.683333
x = 136.916667

query = {'x': (x - .08, x + .1),
         'y': (y - .12, y + 0.03),         
         'crs': 'EPSG:4326'}

# Import ITEM v2.0 relative layer 
item_data = dc.load(product='item_v2', **query)

# Squeeze out the unnecessary temporal dimension:
item_data = item_data.squeeze(dim = 'time', drop = True)

# Filter out -6666 values present in this area
item_data['relative'] = item_data.relative.where(item_data.relative != -6666, 0)

item_data

### Plotting the ITEM Relative Extent data

Using your matplotlib skills you write a function to visualise the extensions exposed at different tide levels.

In [ ]:
def plot_tides(ds):
    # Set up colour schemes and plot labels
    item_cs = ['black', '#d7191c', '#ec6e43', '#fdb96e', '#fee7a4', 
               '#e7f5b7', '#b7e1a7', '#74b6ad', '#2b83ba', 'white']
    labels = ['Non-\nintertidal', '1: 0-10%', '2: 10-20%', '3: 20-30%', 
              '4: 30-40%', '5: 40-50%', '6: 50-60%', '7: 60-70%', '8: 70-80%']

    # Plot ITEM Relative Extent Model
    item_plot = ds.plot(figsize=(8,6), levels=10, colors=item_cs, add_colorbar=False)
    plt.title('ITEM Relative Extent Model')

    # Add tidal interval percentages to colorbar
    cbar = plt.colorbar(item_plot)
    cbar.ax.get_yaxis().set_ticks([])
    cbar.ax.set_title('ITEM interval', loc = 'left')

    for j, label in enumerate(labels):
        cbar.ax.text(1.5, (j + 0.5) / 9.0, label, ha='left', va='center', color='black')
    cbar.ax.get_yaxis().labelpad = 15
    
plot_tides(item_data.relative)

### Masking out the outside of Franklin Harbour

The image returned by DEA contains data outside the Harbour and you are only interested in the data inside. You use the `mask()` function to create a binary mask defined by a polygon with the same extents and resolution of a `DataArray`. This mask is a 2-D numpy array that contains `0` for pixels outside the polygon and `1` for the pixels inside.

In [ ]:
msk = mask(item_data.relative, poly)
plt.imshow(msk)

Now we apply this mask to the ITEM `DataArray` by multiplying (elementwise) the ITEM raster and the mask.

But there is a problem! The area outside the polygon gets assigned a `0`, which indicates "Non-intertidal", and we want this area to be marked with `9` instead. Can you figure out how to achieve this? 

> Tip: Reversing the mask with the logical not numpy `~` operator might be handy.

In [ ]:
### Your code here

#item_data['relative'] = item_data['relative'] * ....


plot_tides(item_data.relative)

In the DEA ITEM documentation you have found an interesting plot representing the correspondance between the `[1-8]` encoding in the product and the actual values of ocean height.

![image](http://dea-public-data.s3-ap-southeast-2.amazonaws.com/ITEM_Intervals/ITEM_REL_209_122.20_-18.07.jpg)

This is very handy and you start to figure out how useful these data is for your project. You decide to create a Python dictionary to map between ITEM values and actual ocean heights.

You then start by computing the total area of the harbour in square meters. 

> Tip: Each pixel represents a square area of 25 meters on the side.

In [ ]:
item_data['relative'] = item_data['relative'].astype(np.float32)
item_data['relative'] = item_data.relative.where(item_data.relative != 9, np.nan)

harbour_area = None

###Solution:

harbour_area

But you realise that not all this area gets filled with water in every tide. The numbers in ITEM represent the level of the sea at which an area gets exposed. 

You make the assumption that each day there are two complete tide cycles. You also estimate an average tide range of 4 meters for Cowell. In low tide the water is 2 meters below the mean sea level and in high tide is 2 meters above.

* The non-intertidal region `0` gets the whole 4 meters of tidal range.
* The areas represented with the largest value `8`, get exposed when the tide goes below +2 meters
* The areas represented with the next value `7`, get exposed when the tide goes below +1.75 meters
* ...
* The last value `1`, gets exposed at the lowest point of the tide, which is -2 meters.

Following this sequence you assign a coefficient to each tidal point representing the proportion of the total 4 meters range that each pixels experience.

The non-intertidal region `0` gets the whole 4 meters of tidal range, so its coefficient is `1`. For the rest of the values you assume a linear decrease in this coefficient mapping the values from `8` to `1` in `0.125` steps.

In [ ]:
item_data['relative'] = item_data.relative.where(item_data.relative != 0, 1)

for i in range(8):
    item_data['relative'] = item_data.relative.where(item_data.relative != i+1, 0.125 + (i*0.125))

d = item_data.relative.data
print(np.nanmax(d))
print(np.nanmin(d))

np.unique(d[~np.isnan(d)])

You are finally ready to get the number of cubic metres *(1000 litres)* that flow in half tidal cycle. *From either hight to low tide and from low to high tide.*


In [ ]:
# water_vol = coeff_sum * 4 metres depth * 25 meters wide * 25 meters height
water_vol = np.nansum(item_data.relative.data) * 4 * 25 * 25

print("The total water volume per half tidal cycle in Franklin Harbour is: {} m^3".format(water_vol))

For converting this number to Watts you need to convert this number into a flow [litres/second].

High tides occur 12 hours and 25 minutes apart. It takes 6 hours and 12.5 minutes for the water at the shore to go from high to low, or from low to high. You decide to use the Datetime Python library to calculate the seconds in 6 hours and 12.5 minutes.

In [ ]:
import datetime

half_cycle_secs = datetime.timedelta(hours=6,minutes=12,seconds=30).total_seconds()

print("The water flow at Franklin Harbour channels is: {} m^3/s".format(water_vol / half_cycle_secs))

Finally you get the formula you've been given to transform this number into actual energy, which consist in dividing the flow by 45.5 to get KWatts.

In [ ]:
print("The Franklin Harbour Tidal Power Plant would generate: {:0.2f} kWatts".format((water_vol / half_cycle_secs) / 45.5))

Congratulations! It's been an intense but productive day of work. You're about to get yourself a deserved coffee when you receive a call from the [Lake Illawarra](https://en.wikipedia.org/wiki/Lake_Illawarra) Authority in NSW. They have heard about your company's new tidal power plant technology and they reckon their location would be the perfect place to test it.

Can you do follow the same process to calculate how much Illawara Tidal Power Plant would generate?